In [1]:
import xarray as xr 
import numpy as np 
import matplotlib.pyplot as plt
import os 
import pandas as pd

In [2]:
dir_in = '/glade/derecho/scratch/wchapman/ADF/f.e.FTORCHmjo_MEANmjo/ts/'
f_in = 'f.e.FTORCHmjo_MEANmjo.cam.h1.FLUT.1979010100000-1993122700000.nc'
var_name = 'FLUT'
file_path = os.path.join(dir_in, f_in)  # Join the directory and file name
DS = xr.open_dataset(file_path)

## Functions

In [3]:
def is_doyrange(doy,dd,tod,hh):
    daywind = 15
    if (dd - daywind) < 1:
        return ((doy >= (366+(dd-daywind))) | (doy <= dd+daywind)) & (tod==hh)
        
    elif (dd + daywind) > 366:
        return ((((doy <= 366) & (doy>=(dd-daywind))) | (doy <= (-1*((366-(dd-daywind))-(2*daywind)))  )))&(tod==hh)
    
    else:
        return ((doy >= dd-daywind) & (doy <= dd+daywind))&(tod==hh)

## Make a climo 
find the first leap year:

In [4]:
def is_leap_year(years):
    """Vectorized check for leap years, suitable for arrays."""
    return (years % 4 == 0) & ((years % 100 != 0) | (years % 400 == 0))

# Using xarray, apply the function across the 'time' coordinate
# First, ensure 'time' is recognized as datetime objects
# Convert time to datetime if not already done
# DS['time'] = xr.DataArray(pd.to_datetime(DS.time.values), dims=['time'])

# Extract year from 'time' coordinate
years = DS['time'].dt.year

# Use the vectorized function to find leap years
leap_year_mask = is_leap_year(years)

# Apply mask to the years data and drop all non-leap years
leap_years = years.where(leap_year_mask, drop=True)

# Extract the first leap year
if len(leap_years) ==0:
    first_leap_year = float(DS['time.year'][0])
else:
    first_leap_year = leap_years.min().values  # Extract the minimum year, which is the first leap year

print("First leap year in the dataset:", first_leap_year)

First leap year in the dataset: 1980.0


In [5]:
%%time
DS_climo = xr.zeros_like(DS.sel(time=slice(f'{int(first_leap_year)}',f'{int(first_leap_year)}'))[var_name])
print(f'... creating climo from centered 30 day average for {var_name}...')
for ee,dayhr in (enumerate(DS.sel(time=slice(f'{int(first_leap_year)}',f'{int(first_leap_year+1)}')).time)):
    if ee%20 ==0:
        print('doing ',ee,' of 366')
    dooDOY = dayhr['time.dayofyear']
    hh=dayhr['time.hour']
    Dtemp = DS.sel(time=is_doyrange(DS['time.dayofyear'],dooDOY,DS['time.hour'],hh))[[var_name]].mean(['time'])
    
    DS_climo[ee,:,:] = Dtemp[var_name].values

    if ee == DS_climo.shape[0]-1:
        endee=ee
        enddate = dayhr
        break

DS_climo.to_dataset().to_netcdf(f'{dir_in}{var_name}_climo.nc')
print('======= make anoms  =======')
varsp=f_in.split(var_name)
anom = DS[var_name].groupby('time.dayofyear')-DS_climo.groupby('time.dayofyear').mean("time")
anom = anom.to_dataset()
anom[var_name].attrs = DS[var_name].attrs
anom.to_netcdf(f'{dir_in}{varsp[0]}{var_name}.anomalies{varsp[-1]}')

... creating climo from centered 30 day average for FLUT...
doing  0  of 366
doing  20  of 366
doing  40  of 366
doing  60  of 366
doing  80  of 366
doing  100  of 366
doing  120  of 366
doing  140  of 366
doing  160  of 366
doing  180  of 366
doing  200  of 366
doing  220  of 366
doing  240  of 366
doing  260  of 366
doing  280  of 366
doing  300  of 366
doing  320  of 366
doing  340  of 366
doing  360  of 366
======= make anoms  =======
CPU times: user 1min 3s, sys: 1min 44s, total: 2min 48s
Wall time: 4min 59s


In [6]:
dir_in = '/glade/derecho/scratch/wchapman/ADF/f.e.FTORCHmjo_MEANmjo/ts/'
f_in = 'f.e.FTORCHmjo_MEANmjo.cam.h1.FLUT.1979010100000-1993122700000.nc'
var_name = 'FLUT'
file_path = os.path.join(dir_in, f_in)  # Join the directory and file name
DS = xr.open_dataset(file_path)

In [12]:
def is_doyrange(doy,dd,tod,hh):
    daywind = 45
    if (dd - daywind) < 1:
        return ((doy >= (366+(dd-daywind))) | (doy <= dd+daywind)) & (tod==hh)
        
    elif (dd + daywind) > 366:
        return ((((doy <= 366) & (doy>=(dd-daywind))) | (doy <= (-1*((366-(dd-daywind))-(2*daywind)))  )))&(tod==hh)
    
    else:
        return ((doy >= dd-daywind) & (doy <= dd+daywind))&(tod==hh)

In [ ]:
is_doyrange(doy,dd,tod,hh)

In [10]:
ttin = pd.to_datetime('2010-04-01')
ttin.hour

0

In [ ]:
time=is_doyrange(DS['time.dayofyear'],dooDOY,DS['time.hour'],hh))[[var_name]].mean(['time']

In [13]:
Dtemp = DS.sel(time=is_doyrange(DS['time.dayofyear'],ttin.dayofyear,DS['time.hour'],ttin.hour))[[var_name]]

In [17]:
len(np.unique(Dtemp['time.dayofyear']))

91